# Evaluate

This notebook evaluates the network trained previous notebooks and analyzes the results.

-----

## Load Packages

In [1]:
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%cd ..
%load_ext autoreload
%autoreload 2

C:\Users\Minjae\Desktop\EEG_Project


In [2]:
# Load some packages
import os
import sys
import pickle
from copy import deepcopy
import hydra
from omegaconf import OmegaConf
from collections import OrderedDict

import numpy as np
from sklearn.metrics import classification_report
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd

import pprint
import wandb
import matplotlib
import matplotlib.pyplot as plt

# custom package
from datasets.caueeg_script import build_dataset_for_train
import models
from train.evaluate import check_accuracy
from train.evaluate import check_accuracy_extended
from train.evaluate import check_accuracy_extended_debug
from train.evaluate import check_accuracy_multicrop
from train.evaluate import check_accuracy_multicrop_extended
from train.evaluate import calculate_class_wise_metrics
from train.visualize import draw_roc_curve
from train.visualize import draw_confusion
from train.visualize import draw_class_wise_metrics
from train.visualize import draw_error_table
from train.visualize import annotate_heatmap

In [3]:
print('PyTorch version:', torch.__version__)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if torch.cuda.is_available(): print('cuda is available.')
else: print('cuda is unavailable.') 

PyTorch version: 1.11.0+cu113
cuda is available.


-----

## List up the models to check accuracy

In [4]:
model_names = [
'2m9vliuk',
'2b2zwwv8',
'22pj8bg1',
'zbnwnfxx',
'1hbsk1hc',
'3smz5tul',
'34oksw2q',
'398v7wpa',
'3ravlttx',
]

model_pool = []

for model_name in model_names:
    path = os.path.join(r'E:\CAUEEG\checkpoint', model_name, 'checkpoint.pt')
    try:
        ckpt = torch.load(path, map_location=device)
        model_pool.append({'name': model_name, 'path': path})
    except Exception as e:
        print(e)
        print(f'- checkpoint cannot be opened: {path}')
        
pprint.pprint([model_dict['name'] for model_dict in model_pool])

['2m9vliuk',
 '2b2zwwv8',
 '22pj8bg1',
 'zbnwnfxx',
 '1hbsk1hc',
 '3smz5tul',
 '34oksw2q',
 '398v7wpa',
 '3ravlttx']


In [5]:
# model_pool = []

# api = wandb.Api()
# runs = api.runs('ipis-mjkim/caueeg-task2-ablation')

# for run in runs:
#     path = os.path.join(r'E:\CAUEEG\checkpoint', run.name, 'checkpoint.pt')
#     try:
#         ckpt = torch.load(path, map_location=device)
#         model_pool.append({'name': run.name, 'path': path})
#     except Exception as e:
#         print(e)
#         print(f'- {run.name}\'s checkpoint cannot be opened: {path}')
        
# pprint.pprint([model_dict['name'] for model_dict in model_pool])

---

## Configurations

In [6]:
base_repeat = 500
verbose = False

-----

## Load and check accuracy

In [7]:
for model_dict in model_pool:
    # load and parse the checkpoint
    ckpt = torch.load(model_dict['path'], map_location=device)
    model_state = ckpt['model_state']
    config = ckpt['config']
    
    model_dict['model'] = config['model']
    model_dict['num_params'] = config.get('num_params', '???')
    model_dict['model size (MiB)'] = sys.getsizeof(pickle.dumps(model_state)) / (1024 * 1024)
    # torch.save(model_state, 'temptemptemp.pt')
    
    model_dict['seq_length'] = config['seq_length']
    model_dict['use_age'] = config['use_age']
    model_dict['photic'] = config['photic']
    model_dict['EKG'] = config['EKG']

    model_dict['awgn'] = config.get('awgn', 0)
    model_dict['awgn_age'] = config.get('awgn_age', 0)
    model_dict['mgn'] = config.get('mgn', 0)
    model_dict['mixup'] = config.get('mixup', 0)
    model_dict['dropout'] = config.get('dropout', 0)
    model_dict['weight_decay'] = config.get('weight_decay', '???')
    model_dict['fc_stages'] = config.get('fc_stages', 1)
    
    model_dict['minibatch'] = round(config['minibatch'])
    model_dict['total_samples'] = round(config.get('total_samples', config['iterations'] * config['minibatch']))
    model_dict['base_lr'] = config.get('base_lr', config.get('LR', '???'))
    model_dict['lr_scheduler_type'] = config.get('lr_scheduler_type', 'constant_with_decay')
    model_dict['warmup_steps'] = config.get('warmup_steps', '???')
    model_dict['seed'] = config.get('seed', '???')
    
    print('- checking for', model_dict['name'], config['model'], '...')
    
    # initiate the model
    if '_target_' in config:
        model = hydra.utils.instantiate(config).to(device)
    elif type(config['generator']) is str:
        config['generator'] = getattr(models, config['generator'].split('.')[-1])
        if 'block' in config:
            config['block'] = getattr(models, config['block'].split('.')[-1])
        model = config['generator'](**config).to(device)
    else:
        if 'block' in config:
            if config['block'] == models.resnet_1d.BottleneckBlock1D:
                config['block'] = 'bottleneck'
            elif config['block'] == models.resnet_2d.Bottleneck2D:
                config['block'] = 'bottleneck'
            elif config['block'] == models.resnet_1d.BasicBlock1D:
                config['block'] = 'basic'
            elif config['block'] == models.resnet_2d.BasicBlock2D:
                config['block'] = 'basic'
                
        model = config['generator'](**config).to(device)
    
    if config.get('ddp', False):
        model_state_ddp = deepcopy(model_state)
        model_state = OrderedDict()
        for k, v in model_state_ddp.items():
            name = k[7:]  # remove 'module.' of DataParallel/DistributedDataParallel
            model_state[name] = v
    
    model.load_state_dict(model_state)
    
    # reconfigure and update
    config.pop('cwd', 0)
    config['ddp'] = False
    config['crop_timing_analysis'] = False
    config['eval'] = True
    config['device'] = device
    
    repeat = round(base_repeat / config['crop_multiple'])
    model_dict['repeat'] = repeat
    model_dict['crop_multiple'] = config['crop_multiple']
    model_dict['test_crop_multiple'] = config['test_crop_multiple']
    model_dict['task'] = config['task']
    
    # build dataset
    _ = build_dataset_for_train(config, verbose=verbose)
    train_loader = _[0]
    val_loader = _[1]
    test_loader = _[2]
    multicrop_test_loader = _[3]
    
    # train accuracy
    train_acc = check_accuracy(model, train_loader, 
                               config['preprocess_test'], config, repeat=repeat)
    model_dict['Train Accuracy'] = train_acc
    
    # val accuracy
    val_acc = check_accuracy(model, val_loader, 
                             config['preprocess_test'], config, repeat=repeat)
    model_dict['Validation Accuracy'] = val_acc
    
    # Test accuracy
    _ = check_accuracy_extended(model, test_loader, 
                                config['preprocess_test'], config, repeat=repeat)
    model_dict['Test Accuracy'] = _[0]
    model_dict['Test Throughput'] = _[4]
    test_class_wise_metrics = calculate_class_wise_metrics(_[3])
    
    for k, v in test_class_wise_metrics.items():
        for c in range(config['out_dims']):
            c_name = config['class_label_to_name'][c]
            model_dict[f'{k} ({c_name})'] = test_class_wise_metrics[k][c]
    
    # Multi-crop test accuracy
    _ = check_accuracy_multicrop_extended(model, multicrop_test_loader, 
                                          config['preprocess_test'], config, repeat=repeat)
    model_dict['Multi-Crop Test Accuracy'] = _[0]
    model_dict['Multi-Crop Test Throughput'] = _[4]
    multi_test_class_wise_metrics = calculate_class_wise_metrics(_[3])
    
    for k, v in multi_test_class_wise_metrics.items():
        for c in range(config['out_dims']):
            c_name = config['class_label_to_name'][c]
            model_dict[f'Multi-Crop {k} ({c_name})'] = multi_test_class_wise_metrics[k][c]
            
print('==== Finished ====')

- checking for 2m9vliuk 2D-Linear-SVM ...
- checking for 2b2zwwv8 2D-Linear-SVM ...
- checking for 22pj8bg1 1D-Linear-SVM ...
- checking for zbnwnfxx 1D-Linear-SVM ...
- checking for 1hbsk1hc Ieracitano-CNN ...
- checking for 3smz5tul Ieracitano-CNN ...
- checking for 34oksw2q Ieracitano-CNN ...
- checking for 398v7wpa Ieracitano-CNN ...
- checking for 3ravlttx 1D-Linear-SVM ...
==== Finished ====


In [8]:
pprint.pprint(model_pool)

[{'Accuracy (Dementia)': 0.713135593220339,
  'Accuracy (MCI)': 0.5987118644067797,
  'Accuracy (Normal)': 0.7355423728813559,
  'EKG': 'O',
  'F1-score (Dementia)': 0.42406506278286316,
  'F1-score (MCI)': 0.36021185753661566,
  'F1-score (Normal)': 0.6976572945530645,
  'Multi-Crop Accuracy (Dementia)': 0.719457627118644,
  'Multi-Crop Accuracy (MCI)': 0.6037966101694915,
  'Multi-Crop Accuracy (Normal)': 0.7589152542372881,
  'Multi-Crop F1-score (Dementia)': 0.4412638401296246,
  'Multi-Crop F1-score (MCI)': 0.347622237106497,
  'Multi-Crop F1-score (Normal)': 0.7292935444579781,
  'Multi-Crop Precision (Dementia)': 0.4627584253752478,
  'Multi-Crop Precision (MCI)': 0.4062092355857031,
  'Multi-Crop Precision (Normal)': 0.6485242350392635,
  'Multi-Crop Recall (Dementia)': 0.4216774193548387,
  'Multi-Crop Recall (MCI)': 0.3038048780487805,
  'Multi-Crop Recall (Normal)': 0.8330434782608696,
  'Multi-Crop Sensitivity (Dementia)': 0.4216774193548387,
  'Multi-Crop Sensitivity (MCI)

In [9]:
pd.DataFrame(model_pool)

,name,path,model,num_params,model size (MiB),seq_length,use_age,photic,EKG,awgn,...,Multi-Crop Specificity (Dementia),Multi-Crop Precision (Normal),Multi-Crop Precision (MCI),Multi-Crop Precision (Dementia),Multi-Crop Recall (Normal),Multi-Crop Recall (MCI),Multi-Crop Recall (Dementia),Multi-Crop F1-score (Normal),Multi-Crop F1-score (MCI),Multi-Crop F1-score (Dementia)
0,2m9vliuk,E:\CAUEEG\checkpoint\2m9vliuk\checkpoint.pt,2D-Linear-SVM,508038,1.938823,2000,fc,O,O,0.00,...,0.825563,0.648524,0.406209,0.462758,0.833043,0.303805,0.421677,0.729294,0.347622,0.441264
1,2b2zwwv8,E:\CAUEEG\checkpoint\2b2zwwv8\checkpoint.pt,2D-Linear-SVM,508038,1.938823,2000,fc,O,O,0.00,...,0.846069,0.652228,0.414492,0.474576,0.824870,0.347122,0.390194,0.728460,0.377827,0.428268
2,22pj8bg1,E:\CAUEEG\checkpoint\22pj8bg1\checkpoint.pt,1D-Linear-SVM,63006,0.241155,1000,fc,O,O,0.00,...,0.767170,0.533693,0.409524,0.359872,0.558537,0.413354,0.334711,0.545832,0.411430,0.346836
3,zbnwnfxx,E:\CAUEEG\checkpoint\zbnwnfxx\checkpoint.pt,1D-Linear-SVM,126006,0.481483,2000,fc,O,O,0.00,...,0.882851,0.650146,0.411237,0.473771,0.814435,0.411317,0.296000,0.723076,0.411277,0.364358
4,1hbsk1hc,E:\CAUEEG\checkpoint\1hbsk1hc\checkpoint.pt,Ieracitano-CNN,3457995,13.198660,1000,no,X,X,0.00,...,0.876805,0.598785,0.437127,0.557107,0.762826,0.375268,0.434903,0.670924,0.403843,0.488478
5,3smz5tul,E:\CAUEEG\checkpoint\3smz5tul\checkpoint.pt,Ieracitano-CNN,3457995,13.198660,1000,no,X,X,0.00,...,0.919859,0.724747,0.698096,0.779068,0.774816,0.691223,0.722639,0.748945,0.694643,0.749793
6,34oksw2q,E:\CAUEEG\checkpoint\34oksw2q\checkpoint.pt,Ieracitano-CNN,3457995,13.198660,1000,no,X,X,0.00,...,0.993907,0.967001,0.928844,0.983789,0.950748,0.970824,0.945520,0.958806,0.949370,0.964275
7,398v7wpa,E:\CAUEEG\checkpoint\398v7wpa\checkpoint.pt,Ieracitano-CNN,3457995,13.198660,1000,no,X,X,0.00,...,0.876506,0.636353,0.509565,0.644897,0.941913,0.244293,0.629419,0.759554,0.330256,0.637064
8,3ravlttx,E:\CAUEEG\checkpoint\3ravlttx\checkpoint.pt,1D-Linear-SVM,120006,0.458594,2000,fc,X,O,0.05,...,0.892874,0.648664,0.422412,0.485424,0.814609,0.433951,0.283613,0.722227,0.428104,0.358039


In [10]:
pd.DataFrame(model_pool).to_csv('local/output/caueeg-noname.csv')